In [1]:
import pandas as pd
import pyrankvote
from pathlib import Path
from random import seed
seed(252)

In [2]:
df = pd.read_csv(Path('./data/20210515.csv').resolve())
df

,Timestamp,Rank your movie choices. You don't have to do a full ranking. [First Choice],Rank your movie choices. You don't have to do a full ranking. [Second Choice],Rank your movie choices. You don't have to do a full ranking. [Third Choice],Rank your movie choices. You don't have to do a full ranking. [Fourth Choice],What theme should we explore next week? [First Choice],What theme should we explore next week? [Second Choice],What theme should we explore next week? [Third Choice],What theme should we explore next week? [Fourth Choice],What theme should we explore next week? [Fifth Choice]
0,11/05/2021 17:45:46,Breaker Morant (1980),The Browning Version (1951),How To Get Ahead In Advertising (1989),Who’s Afraid of Virginia Woolf? (1966),HIGH FANTASY,[Default to Nicolas Cage],FAMOUS ELEVATORS,FOOD: FRIEND OR FOE?!,LADIES’ NIGHT
1,11/05/2021 18:02:27,The Browning Version (1951),Breaker Morant (1980),How To Get Ahead In Advertising (1989),NaN,FAMOUS ELEVATORS,LADIES’ NIGHT,HIGH FANTASY,[Default to Nicolas Cage],FOOD: FRIEND OR FOE?!
2,11/05/2021 19:32:33,The Browning Version (1951),How To Get Ahead In Advertising (1989),Who’s Afraid of Virginia Woolf? (1966),Breaker Morant (1980),FOOD: FRIEND OR FOE?!,HIGH FANTASY,LADIES’ NIGHT,FAMOUS ELEVATORS,[Default to Nicolas Cage]
3,11/05/2021 19:40:43,Who’s Afraid of Virginia Woolf? (1966),Breaker Morant (1980),NaN,NaN,LADIES’ NIGHT,[Default to Nicolas Cage],NaN,NaN,NaN
4,11/05/2021 20:43:41,How To Get Ahead In Advertising (1989),The Browning Version (1951),Who’s Afraid of Virginia Woolf? (1966),Breaker Morant (1980),[Default to Nicolas Cage],LADIES’ NIGHT,NaN,FOOD: FRIEND OR FOE?!,HIGH FANTASY
5,12/05/2021 05:23:36,The Browning Version (1951),Who’s Afraid of Virginia Woolf? (1966),How To Get Ahead In Advertising (1989),Breaker Morant (1980),LADIES’ NIGHT,FAMOUS ELEVATORS,FOOD: FRIEND OR FOE?!,HIGH FANTASY,[Default to Nicolas Cage]


In [3]:
just_ballots = df.loc[:, df.columns.str.startswith('Rank')]
next_week = df.loc[:, df.columns.str.startswith('What')]

In [4]:
submissions = just_ballots.loc[just_ballots.notna().all(axis=1).idxmax(),:].values
submissions

array(['Breaker Morant (1980)', 'The Browning Version (1951)',
       'How To Get Ahead In Advertising (1989)',
       'Who’s Afraid of Virginia Woolf? (1966)'], dtype=object)

In [5]:
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}

In [6]:
balloted = just_ballots.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)

In [7]:
submission_dict

{'Breaker Morant (1980)': <Candidate('Breaker Morant (1980)')>,
 'The Browning Version (1951)': <Candidate('The Browning Version (1951)')>,
 'How To Get Ahead In Advertising (1989)': <Candidate('How To Get Ahead In Advertising (1989)')>,
 'Who’s Afraid of Virginia Woolf? (1966)': <Candidate('Who’s Afraid of Virginia Woolf? (1966)')>}

In [8]:
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))

In [9]:
ballots

[<Ballot(Breaker Morant (1980), The Browning Version (1951), How To Get Ahead In Advertising (1989), Who’s Afraid of Virginia Woolf? (1966))>,
 <Ballot(The Browning Version (1951), Breaker Morant (1980), How To Get Ahead In Advertising (1989))>,
 <Ballot(The Browning Version (1951), How To Get Ahead In Advertising (1989), Who’s Afraid of Virginia Woolf? (1966), Breaker Morant (1980))>,
 <Ballot(Who’s Afraid of Virginia Woolf? (1966), Breaker Morant (1980))>,
 <Ballot(How To Get Ahead In Advertising (1989), The Browning Version (1951), Who’s Afraid of Virginia Woolf? (1966), Breaker Morant (1980))>,
 <Ballot(The Browning Version (1951), Who’s Afraid of Virginia Woolf? (1966), How To Get Ahead In Advertising (1989), Breaker Morant (1980))>]

In [10]:
election_result = pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,)

In [11]:
print(election_result)

ROUND 1
Candidate                                 Votes  Status
--------------------------------------  -------  --------
The Browning Version (1951)                   3  Hopeful
How To Get Ahead In Advertising (1989)        1  Hopeful
Breaker Morant (1980)                         1  Rejected
Who’s Afraid of Virginia Woolf? (1966)        1  Rejected

FINAL RESULT
Candidate                                 Votes  Status
--------------------------------------  -------  --------
The Browning Version (1951)                   4  Elected
How To Get Ahead In Advertising (1989)        1  Rejected
Breaker Morant (1980)                         0  Rejected
Who’s Afraid of Virginia Woolf? (1966)        0  Rejected
Blank votes                                   1  Rejected



In [12]:
submissions = next_week.loc[next_week.notna().all(axis=1).idxmax(),:].values
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}
balloted = next_week.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))
print(pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,))

ROUND 1
Candidate                    Votes  Status
-------------------------  -------  --------
LADIES’ NIGHT                    2  Hopeful
[Default to Nicolas Cage]        1  Hopeful
HIGH FANTASY                     1  Hopeful
FAMOUS ELEVATORS                 1  Hopeful
FOOD: FRIEND OR FOE?!            1  Rejected

ROUND 2
Candidate                    Votes  Status
-------------------------  -------  --------
LADIES’ NIGHT                    2  Hopeful
HIGH FANTASY                     2  Hopeful
[Default to Nicolas Cage]        1  Rejected
FAMOUS ELEVATORS                 1  Rejected
FOOD: FRIEND OR FOE?!            0  Rejected

FINAL RESULT
Candidate                    Votes  Status
-------------------------  -------  --------
LADIES’ NIGHT                    4  Elected
HIGH FANTASY                     2  Rejected
[Default to Nicolas Cage]        0  Rejected
FAMOUS ELEVATORS                 0  Rejected
FOOD: FRIEND OR FOE?!            0  Rejected

